In [1]:
""" # Instalar a última versão do PySpark
!pip install pyspark #==3.3.1

# Instalar o NGROK
!wget -qnc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -n -q ngrok-stable-linux-amd64.zip
 """

# Iniciar a sessão spark
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder                  
      .config('spark.ui.port', '4050')
      .appName("SparkSQL")
      .getOrCreate()
)

# Autenticar a sessão do SparkUI com NGROK
!./ngrok authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT
get_ipython().system_raw('./ngrok http 4050 &')
!sleep 10
!curl -s http://localhost:4040/api/tunnels | grep -Po 'public_url":"(?=https)\K[^"]*'

23/10/17 21:22:10 WARN Utils: Your hostname, valdir-G5-5590 resolves to a loopback address: 127.0.1.1; using 192.168.0.43 instead (on interface wlp4s0)
23/10/17 21:22:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/17 21:22:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/bin/bash: line 1: ./ngrok: No such file or directory


/bin/bash: line 1: ./ngrok: No such file or directory


In [2]:
""" from google.colab import drive
drive.mount('/content/drive') """

" from google.colab import drive\ndrive.mount('/content/drive') "

In [3]:
from pyspark.sql.types import *

schema_remetente_destinatario = StructType([
    StructField('nome', StringType()),
    StructField('banco', StringType()),
    StructField('tipo', StringType())
])

schema_base_pix = StructType([
    StructField('id_transacao', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('remetente', schema_remetente_destinatario),
    StructField('destinatario', schema_remetente_destinatario),
    StructField('chave_pix', StringType()),
    StructField('categoria', StringType()),
    StructField('transaction_date', StringType()),
    StructField('fraude', IntegerType())
])

caminho_json = 'case_final.json'

df = spark.read.json(
    caminho_json,
    schema=schema_base_pix,
    timestampFormat="yyyy-MM-dd HH:mm:ss"
)

In [4]:
df.printSchema()

df.show()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- remetente: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- destinatario: struct (nullable = true)
 |    |-- nome: string (nullable = true)
 |    |-- banco: string (nullable = true)
 |    |-- tipo: string (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)



+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|id_transacao|             valor|           remetente|        destinatario|chave_pix|    categoria|   transaction_date|fraude|
+------------+------------------+--------------------+--------------------+---------+-------------+-------------------+------+
|        1000|            588.08|{Jonathan Gonsalv...|{Calebe Melo, Cai...|aleatoria|       outros|2021-07-16 05:00:55|     0|
|        1001|           80682.5|{Jonathan Gonsalv...|{Davi Lucas Perei...|  celular|transferencia|2022-04-20 12:34:01|     1|
|        1002|             549.9|{Jonathan Gonsalv...|{Sabrina Castro, ...|      cpf|        lazer|2022-07-10 16:51:34|     0|
|        1003|             90.83|{Jonathan Gonsalv...|{Francisco da Con...|aleatoria|   transporte|2022-10-20 10:57:36|     0|
|        1004|13272.619999999999|{Jonathan Gonsalv...|{Isabelly Ferreir...|    email|transferencia|2021-04-06 2

In [5]:
from pyspark.sql.functions import *

df_flatten = df.withColumns({
    'destinatario_nome': col('destinatario').getField('nome'),
    'destinatario_banco': col('destinatario').getField('banco'),
    'destinatario_tipo': col('destinatario').getField('tipo'),
}).drop('remetente', 'destinatario')

In [6]:
df_flatten.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)
 |-- destinatario_nome: string (nullable = true)
 |-- destinatario_banco: string (nullable = true)
 |-- destinatario_tipo: string (nullable = true)



In [7]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(
    inputCols=[
        'destinatario_nome',
        'destinatario_banco',
        'destinatario_tipo',
        'categoria',
        'chave_pix'
    ],
    outputCols=[
        'destinatario_nome_index',
        'destinatario_banco_index',
        'destinatario_tipo_index',
        'categoria_index',
        'chave_pix_index'        
    ]
)

df_index = indexer.fit(df_flatten).transform(df_flatten)

In [8]:
df_index.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)
 |-- destinatario_nome: string (nullable = true)
 |-- destinatario_banco: string (nullable = true)
 |-- destinatario_tipo: string (nullable = true)
 |-- destinatario_nome_index: double (nullable = false)
 |-- destinatario_banco_index: double (nullable = false)
 |-- destinatario_tipo_index: double (nullable = false)
 |-- categoria_index: double (nullable = false)
 |-- chave_pix_index: double (nullable = false)



In [9]:
df_index.show()

23/10/17 21:22:54 WARN DAGScheduler: Broadcasting large task binary with size 1276.9 KiB


+------------+------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|id_transacao|             valor|chave_pix|    categoria|   transaction_date|fraude|   destinatario_nome|destinatario_banco|destinatario_tipo|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|categoria_index|chave_pix_index|
+------------+------------------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+
|        1000|            588.08|aleatoria|       outros|2021-07-16 05:00:55|     0|         Calebe Melo|             Caixa|               PF|                12045.0|                     4.0|                    1.0|            6.0|            3.0|
|       

In [10]:
is_fraud = df_index.filter("fraude == 1") # filter(col('fraude') == 1)
no_fraud = df_index.filter("fraude == 0")

In [11]:
no_fraud = no_fraud.sample(False, 0.01, seed=123) 

In [12]:
df_concat = no_fraud.union(is_fraud)
df = df_concat.sort("transaction_date")
df.count()

16202

In [13]:
# Usa randomSplit com seed para garantir a mesma divisão.
# O seed é opcional e é usado para melhorar a coleta aleatória dos dados.
train, test = df.randomSplit([0.7, 0.3], seed = 123)
print("train =", train.count(), " test =", test.count())

23/10/17 21:23:03 WARN DAGScheduler: Broadcasting large task binary with size 1294.3 KiB
23/10/17 21:23:10 WARN DAGScheduler: Broadcasting large task binary with size 1294.8 KiB
23/10/17 21:23:16 WARN DAGScheduler: Broadcasting large task binary with size 1306.5 KiB
23/10/17 21:23:23 WARN DAGScheduler: Broadcasting large task binary with size 1297.5 KiB
23/10/17 21:23:24 WARN DAGScheduler: Broadcasting large task binary with size 1294.3 KiB
23/10/17 21:23:28 WARN DAGScheduler: Broadcasting large task binary with size 1294.8 KiB
23/10/17 21:23:31 WARN DAGScheduler: Broadcasting large task binary with size 1306.5 KiB
23/10/17 21:23:34 WARN DAGScheduler: Broadcasting large task binary with size 1297.5 KiB


train = 11278  test = 4924


In [14]:
# testar para ver se a divisão dos dados foi 70 e 30
print("train =", train.count() / df.count(), " test =", test.count() / df.count())

23/10/17 21:23:35 WARN DAGScheduler: Broadcasting large task binary with size 1294.3 KiB
23/10/17 21:23:38 WARN DAGScheduler: Broadcasting large task binary with size 1294.8 KiB
23/10/17 21:23:40 WARN DAGScheduler: Broadcasting large task binary with size 1306.5 KiB
23/10/17 21:23:44 WARN DAGScheduler: Broadcasting large task binary with size 1297.5 KiB
23/10/17 21:23:46 WARN DAGScheduler: Broadcasting large task binary with size 1294.3 KiB
23/10/17 21:23:49 WARN DAGScheduler: Broadcasting large task binary with size 1294.8 KiB
23/10/17 21:23:52 WARN DAGScheduler: Broadcasting large task binary with size 1306.5 KiB
23/10/17 21:23:55 WARN DAGScheduler: Broadcasting large task binary with size 1297.5 KiB


train = 0.6960869028514998  test = 0.3039130971485002


In [15]:
# o UDF é uma função que podemos criar para aplicar em uma coluna
is_fraud = udf(lambda fraud: 1.0 if fraud > 0 else 0.0, DoubleType())
train = train.withColumn("is_fraud", is_fraud(train.fraude))

In [16]:
train.printSchema()

root
 |-- id_transacao: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- chave_pix: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- transaction_date: string (nullable = true)
 |-- fraude: integer (nullable = true)
 |-- destinatario_nome: string (nullable = true)
 |-- destinatario_banco: string (nullable = true)
 |-- destinatario_tipo: string (nullable = true)
 |-- destinatario_nome_index: double (nullable = false)
 |-- destinatario_banco_index: double (nullable = false)
 |-- destinatario_tipo_index: double (nullable = false)
 |-- categoria_index: double (nullable = false)
 |-- chave_pix_index: double (nullable = false)
 |-- is_fraud: double (nullable = true)



In [17]:
train.columns

['id_transacao',
 'valor',
 'chave_pix',
 'categoria',
 'transaction_date',
 'fraude',
 'destinatario_nome',
 'destinatario_banco',
 'destinatario_tipo',
 'destinatario_nome_index',
 'destinatario_banco_index',
 'destinatario_tipo_index',
 'categoria_index',
 'chave_pix_index',
 'is_fraud']

In [18]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline

assembler = VectorAssembler(
    inputCols = [x for x in train.columns if x not in ['transaction_date', 'fraude', 'is_fraud', 'destinatario_nome', 'destinatario_banco', 'destinatario_tipo', 'chave_pix', 'categoria']],
    outputCol="features"
)

In [19]:
lr = LogisticRegression().setParams(
    maxIter=100000,
    labelCol = "is_fraud",
    predictionCol="prediction"
)

In [20]:
model = Pipeline(stages=[assembler, lr]).fit(train)

23/10/17 21:23:59 WARN DAGScheduler: Broadcasting large task binary with size 1294.3 KiB
23/10/17 21:24:02 WARN DAGScheduler: Broadcasting large task binary with size 1294.8 KiB
23/10/17 21:24:05 WARN DAGScheduler: Broadcasting large task binary with size 1306.5 KiB
23/10/17 21:24:10 WARN DAGScheduler: Broadcasting large task binary with size 1294.3 KiB
23/10/17 21:24:15 WARN DAGScheduler: Broadcasting large task binary with size 1294.8 KiB
23/10/17 21:24:17 WARN DAGScheduler: Broadcasting large task binary with size 1306.5 KiB
23/10/17 21:24:20 WARN DAGScheduler: Broadcasting large task binary with size 1343.0 KiB
23/10/17 21:24:27 WARN DAGScheduler: Broadcasting large task binary with size 1343.7 KiB
23/10/17 21:24:28 WARN DAGScheduler: Broadcasting large task binary with size 1343.7 KiB
23/10/17 21:24:28 WARN DAGScheduler: Broadcasting large task binary with size 1343.7 KiB
23/10/17 21:24:28 WARN DAGScheduler: Broadcasting large task binary with size 1343.7 KiB
23/10/17 21:24:29 WAR

In [21]:
predicted = model.transform(test)

In [22]:
predicted.show()

23/10/17 21:24:55 WARN DAGScheduler: Broadcasting large task binary with size 1294.3 KiB
23/10/17 21:24:59 WARN DAGScheduler: Broadcasting large task binary with size 1294.8 KiB
23/10/17 21:25:01 WARN DAGScheduler: Broadcasting large task binary with size 1306.5 KiB
23/10/17 21:25:04 WARN DAGScheduler: Broadcasting large task binary with size 1342.7 KiB


+------------+--------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+--------------------+--------------------+--------------------+----------+
|id_transacao|   valor|chave_pix|    categoria|   transaction_date|fraude|   destinatario_nome|destinatario_banco|destinatario_tipo|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|categoria_index|chave_pix_index|            features|       rawPrediction|         probability|prediction|
+------------+--------+---------+-------------+-------------------+------+--------------------+------------------+-----------------+-----------------------+------------------------+-----------------------+---------------+---------------+--------------------+--------------------+--------------------+----------+
|        1011|21345.91|      cpf|transferencia|2021-10-31 04:31:

In [23]:
predicted = predicted.withColumn('is_fraud', is_fraud(predicted.fraude))
predicted.crosstab('is_fraud', 'prediction').show()

23/10/17 21:25:06 WARN DAGScheduler: Broadcasting large task binary with size 1294.3 KiB
23/10/17 21:25:10 WARN DAGScheduler: Broadcasting large task binary with size 1294.8 KiB
23/10/17 21:25:12 WARN DAGScheduler: Broadcasting large task binary with size 1306.5 KiB
23/10/17 21:25:17 WARN DAGScheduler: Broadcasting large task binary with size 1341.9 KiB
23/10/17 21:25:18 WARN DAGScheduler: Broadcasting large task binary with size 1332.0 KiB
23/10/17 21:25:18 WARN DAGScheduler: Broadcasting large task binary with size 1331.7 KiB
23/10/17 21:25:18 WARN DAGScheduler: Broadcasting large task binary with size 1329.4 KiB
23/10/17 21:25:19 WARN DAGScheduler: Broadcasting large task binary with size 1294.3 KiB
23/10/17 21:25:21 WARN DAGScheduler: Broadcasting large task binary with size 1294.8 KiB
23/10/17 21:25:23 WARN DAGScheduler: Broadcasting large task binary with size 1306.5 KiB
23/10/17 21:25:26 WARN DAGScheduler: Broadcasting large task binary with size 1348.7 KiB
23/10/17 21:25:27 WAR

+-------------------+---+----+
|is_fraud_prediction|0.0| 1.0|
+-------------------+---+----+
|                1.0|  0|4682|
|                0.0|242|   0|
+-------------------+---+----+



23/10/17 21:25:27 WARN DAGScheduler: Broadcasting large task binary with size 1339.1 KiB


In [24]:
df_teste_cols = [
    'id_transacao',
    'valor', 
    'transaction_date',
    'destinatario_nome_index',
    'destinatario_banco_index',
    'destinatario_tipo_index',
    'chave_pix_index',
    'categoria_index',
    'fraude'
]

df_teste_data = [
    (999,103.2, "2023-01-01 11:56:41", 328.0, 4.0, 1.0, 3.0, 5.0, 0),
    (998, 500000.0, "2023-01-01 11:56:41", 328.0, 2.0, 3.0, 2.0, 5.0, 1),
    (997, 19999.0, "2023-01-01 11:56:41", 328.0, 1.0, 2.0, 1.0, 5.0, 0),
]

df_teste = spark.createDataFrame(df_teste_data).toDF(*df_teste_cols)

In [25]:
new_prediction = model.transform(df_teste)

In [26]:
new_prediction.show()

+------------+--------+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+--------------------+--------------------+-----------+----------+
|id_transacao|   valor|   transaction_date|destinatario_nome_index|destinatario_banco_index|destinatario_tipo_index|chave_pix_index|categoria_index|fraude|            features|       rawPrediction|probability|prediction|
+------------+--------+-------------------+-----------------------+------------------------+-----------------------+---------------+---------------+------+--------------------+--------------------+-----------+----------+
|         999|   103.2|2023-01-01 11:56:41|                  328.0|                     4.0|                    1.0|            3.0|            5.0|     0|[999.0,103.2,328....|[426.491663445979...|  [1.0,0.0]|       0.0|
|         998|500000.0|2023-01-01 11:56:41|                  328.0|                     2.0|                    3.0|

23/10/17 22:37:30 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 3800088 ms exceeds timeout 120000 ms
23/10/17 22:37:30 WARN SparkContext: Killing executors is not supported by current scheduler.
23/10/17 22:37:34 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$